In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import seaborn as sns
import os
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2022-09-18 13:31:33--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv.1’

housing.csv.1       100%[===================>]   1.36M  --.-KB/s    in 0.07s   

2022-09-18 13:31:33 (19.7 MB/s) - ‘housing.csv.1’ saved [1423529/1423529]



In [33]:
df_copy= df.copy()

df = os.listdir()[1]
df = pd.read_csv(df)
df.drop(['ocean_proximity'], axis=1, inplace=True)
df = shuffle(df)
df.reset_index(inplace=True, drop=True) 


In [3]:
'''Question 1
Find a feature with missing values. How many missing values does it have?

'''
'''
Question 2
What's the median (50% percentile) for variable 'population'?

'''

"\nQuestion 2\nWhat's the median (50% percentile) for variable 'population'?\n\n"

In [4]:
#Split the data

In [5]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [16]:
n = len(df)
n_val = int(n*0.2)
n_test = int(n*0.2)
n_train = n - n_val - n_test
print(n, n_val, n_test, n_train)

20640 4128 4128 12384


In [17]:
idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)

df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]


df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']



In [19]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]


def prepare_X_zero(df):
    df_num = df
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

def prepare_X_mean(df):
    df_num = df
    mean = df['total_bedrooms'].mean()
    df_num = df_num.fillna(mean)
    X = df_num.values
    return X    
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return round(np.sqrt(mse), 2)

def prepare_X_mean_val(df,mean):
    df_num = df
    df_num = df_num.fillna(mean)
    X = df_num.values
    return X


In [14]:
X_train = prepare_X_zero(df_train)
w_0, w = train_linear_regression(X_train, y_train)
X_val = prepare_X_zero(df_val)
y_pred_zero = w_0 + X_val.dot(w)


In [15]:
rmse(y_val, y_pred_zero)

0.35

In [23]:
X_train = prepare_X_mean(df_train)
w_0, w = train_linear_regression(X_train, y_train)

X_val = prepare_X_mean_val(df_val,df_train['total_bedrooms'].mean())
y_pred_mean = w_0 + X_val.dot(w)


In [24]:
rmse(y_val, y_pred_mean)

0.34

In [4]:
#              Q4

In [25]:
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [26]:
X_train = prepare_X_zero(df_train)


In [30]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
    X_val = prepare_X_zero(df_val)
    y_pred = w_0 + X_val.dot(w)
    print('%2s, %.2f' % (r,rmse(y_val, y_pred)))


 0, 0.35
1e-06, 0.35
0.0001, 0.35
0.001, 0.35
0.01, 0.35
0.1, 0.35
 1, 0.35
 5, 0.36
10, 0.36


In [4]:
#           Q5

In [31]:
def rmse_seed(df, seed):

    n = len(df)
    idx = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(idx)

    df["idx"] = idx
    df = df.sort_values(by="idx", ascending=True)
    df = df.drop(columns=["idx"])
    df = df.reset_index(drop=True)

    n_val = int(0.2 * n)
    n_test = int(0.2 * n)
    n_train = n - (n_val + n_test)
    df_train = df.iloc[:n_train]
    df_val = df.iloc[n_train : n_train + n_val]
    df_test = df.iloc[n_train + n_val :]

    y_train = np.log1p(df_train.median_house_value.values)
    y_val = np.log1p(df_val.median_house_value.values)

    del df_train['median_house_value']
    del df_val['median_house_value']

    x_train = prepare_X_zero(df_train)
    w0, w = train_linear_regression(x_train, y_train)
    
    x_val = prepare_X_zero(df_val)
    y_pred =  w0 + x_val.dot(w)
    
    rmse_value = rmse(y_pred, y_val)
    return rmse_value


In [36]:
rmse_list = []
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for seed in seeds:
    rmse_value = rmse_seed(df_copy, seed)
    rmse_list.append(rmse_value)
    print('Seed = %s, %f' % (seed, rmse_value))

standard_deviation = round(np.std(rmse_list),3)
print(standard_deviation) 


Seed = 0, 0.340000
Seed = 1, 0.340000
Seed = 2, 0.340000
Seed = 3, 0.350000
Seed = 4, 0.350000
Seed = 5, 0.340000
Seed = 6, 0.340000
Seed = 7, 0.350000
Seed = 8, 0.340000
Seed = 9, 0.340000
0.005


In [4]:
#          Q6

In [37]:
def rmse_seed_Q_6(df, seed):

    n = len(df)
    idx = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(idx)

    df["idx"] = idx
    df = df.sort_values(by="idx", ascending=True)
    df = df.drop(columns=["idx"])
    df = df.reset_index(drop=True)

    n_test = int(0.2 * n)
    n_train = n - n_test
    df_train = df.iloc[:n_train]
    df_test = df.iloc[n_train:]

    y_train = np.log1p(df_train.median_house_value.values)
    y_test = np.log1p(df_test.median_house_value.values)

    del df_train['median_house_value']
    del df_test['median_house_value']

    x_train = prepare_X_zero(df_train)
    w0, w = train_linear_regression_reg(x_train, y_train, 0.001)
    
    x_test = prepare_X_zero(df_test)
    y_pred =  w0 + x_test.dot(w)
    
    rmse_value = rmse(y_pred, y_test)
    return rmse_value

In [38]:
rmse_seed_Q_6(df_copy, 9)

0.34